In [2]:
import torch
from torch.utils.data import DataLoader

import cellshape_cloud as cscloud
from cellshape_cloud.pointcloud_dataset import PointCloudDataset, SingleCellDataset
from cellshape_cloud.cloud_autoencoder import CloudAutoEncoder
from cellshape_cloud.vendor.chamfer_distance import ChamferLoss

import cellshape_cluster as cscluster
from cellshape_cluster.deep_embedded_clustering import DeepEmbeddedClustering

In [3]:
root_dir = '/home/mvries/Documents/Datasets/'
path = '/run/user/1128299809/gvfs/smb-share:server=rds.icr.ac.uk,share=data/DBI/DUDBI/DYNCESYS/mvries/ResultsAlma/TearingNetNew/DEC/nets/dgcnn_foldingnetDECAll_128_005.pt'

OSError: [Errno 22] Invalid argument: '/run/user/1128299809/gvfs/smb-share:server=rds.icr.ac.uk,share=data/DBI/DUDBI/DYNCESYS/mvries/ResultsAlma/TearingNetNew/DEC/nets/dgcnn_foldingnetDECAll_128_005.pt'